In [44]:
let newline = print_newline;;
let print_int i = Printf.printf "%d "  i;;

val newline : unit -> unit = <fun>


val print_int : int -> unit = <fun>


In [45]:
(* first class module *)
module type X_int = sig val x : int end;;
module One:X_int = struct let x = 1 end;;
module Two = (struct let x = 2 end : X_int);;
module Three = struct let x = 3 end;;

let one = (module One: X_int);;
let two = (module Two:X_int);;
let three = (module Three:X_int);;
let four = (module struct let x = 4 end : X_int);;

let to_int (module M : X_int) = M.x;;
(* let to_int m = let module M = (val m: X_int) in M.x *)

let numbers = [one; two; three; four; (module struct let x = 5 end : X_int)];;
numbers |> List.iter (fun m -> print_int (to_int m)); newline();;

let plus m1 m2 = (module struct let x = to_int m1 + to_int m2 end : X_int);;
print_int (to_int (plus one (module struct let x = 0 end : X_int))); 
newline();;

let sum = numbers |> List.fold_left (fun init m -> plus init m) (module struct let x = 0 end: X_int);;
print_int (to_int sum);;
newline();;

module type X_int = sig val x : int end


module One : X_int


module Two : X_int


module Three : sig val x : int end


val one : (module X_int) = <module>


...

- : unit = ()


val sum : (module X_int) = <module>


- : unit = ()


15 


- : unit = ()


In [46]:
(* first class module with locally abstract type *)
module type Bumpable = sig
    type t
    val bump: t -> t
end;;

module Int_bumper = struct 
    type t = int
    let bump n = n + 1
end;;

module Float_bumper = struct
    type t = float
    let bump n = n +. 1.
end;;

let int_bumper = (module Int_bumper: Bumpable with type t = int);; 
let (module Bumper) = int_bumper in Bumper.bump 1;;
let bump m = let module Bumper = (val m : Bumpable with type t = int) in Bumper.bump 2;;
bump int_bumper;;

let bump (type a) (module Bumper: Bumpable with type t = a) n =  Bumper.bump n;;
bump int_bumper 11;;

let bump_list (type a) (module Bumper: Bumpable with type t = a) (l: a list) =
    l |> List.map (fun x -> Bumper.bump x);;
bump_list int_bumper [1;2;3;4;5] |> List.iter print_int;;
newline();;

module type Bumpable = sig type t val bump : t -> t end


module Int_bumper : sig type t = int val bump : int -> int end


module Float_bumper : sig type t = float val bump : float -> float end


val int_bumper : (module Bumpable with type t = int) = <module>


- : int = 2


...

- : int = 12


val bump_list : (module Bumpable with type t = 'a) -> 'a list -> 'a list =
  <fun>


- : unit = ()


2 3 4 5 6 


- : unit = ()


In [47]:
module type Comparable = sig
    type t
    val compare: t -> t -> int
end;;

let create_compare (type a) cmp =
    (module struct 
        type t = a
        let compare = cmp
    end : Comparable with type t = a);;

let int_compare = create_compare Int.compare;;

module type Comparable = sig type t val compare : t -> t -> int end


val create_compare :
  ('a -> 'a -> int) -> (module Comparable with type t = 'a) = <fun>


val int_compare : (module Comparable with type t = int) = <module>


In [48]:
module type M = sig
     type 'a t
     val x : 'a t
end
 
module ListM : (M with type 'a t = 'a list) = struct
     type 'a t = 'a list
     let x : 'a t = []
end

module type M = sig type 'a t val x : 'a t end


module ListM : sig type 'a t = 'a list val x : 'a t end
